<a href="https://colab.research.google.com/github/dbtmddn41/deep_learning/blob/main/translater.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive')

# TODO: Enter the foldername in your Drive where you have saved the unzipped
# assignment folder, e.g. 'cs231n/assignments/assignment1/'
FOLDERNAME = '딥러닝/keras_creator/Lecture\ 11.'

# Now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

%cd /content/drive/My\ Drive/$FOLDERNAME/

Mounted at /content/drive
/content/drive/My Drive/딥러닝/keras_creator/Lecture 11.


In [ ]:
import string
import re
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.layers import TextVectorization

!pip install konlpy

from konlpy.tag import Okt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 68.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 39.8 MB/s eta 0:00:00


In [ ]:
strip_chars = string.punctuation
strip_chars = strip_chars.replace('[', '')
strip_chars = strip_chars.replace(']', '')
okt = Okt()

def target_standardization(input_string):
    lower_case = tf.strings.lower(input_string)
    return tf.strings.regex_replace(lower_case, f"[{re.escape(strip_chars)}]", '')

vocab_size = 15000
sequence_len = 35

korean_vectorization = layers.TextVectorization(max_tokens=vocab_size, output_mode='int', output_sequence_length=sequence_len)
english_vectorization = layers.TextVectorization(max_tokens=vocab_size, output_mode='int', output_sequence_length=sequence_len+1, standardize=target_standardization)

import pickle
with open('kor_vocab.pkl', 'rb') as f:
    kor_vocab = pickle.load(f)
    korean_vectorization.set_vocabulary(kor_vocab)

with open('eng_vocab.pkl', 'rb') as f:
    eng_vocab = pickle.load(f)
    english_vectorization.set_vocabulary(eng_vocab)

In [ ]:
class TransformerEncoderBlock(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, dropout_rate=0.0, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.dropout_rate = dropout_rate

        self.attention = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim, dropout=self.dropout_rate)
        self.dense_proj = keras.Sequential([layers.Dense(dense_dim, activation='relu'), layers.Dense(embed_dim)])
        self.dropout1 = layers.Dropout(dropout_rate)
        self.dropout2 = layers.Dropout(dropout_rate)
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(query=inputs, value=inputs, attention_mask=mask)
        x = inputs + self.dropout1(attention_output)
        x = self.layernorm_1(x)
        proj_output = self.dense_proj(x)
        x = self.dropout2(proj_output) + x
        output = self.layernorm_2(x)

        return output

    def get_config(self):
        config = super().get_config()
        config.update({"embed_dim": self.embed_dim, "dense_dim": self.dense_dim, "num_heads": self.num_heads, 'dropout_rate': self.dropout_rate})
        return config

class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_len, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(input_dim=input_dim, output_dim=output_dim)#, embeddings_initializer=keras.initializers.Constant(embedding_matrix), trainable=False)
        self.position_embeddings = layers.Embedding(input_dim=sequence_len, output_dim=output_dim)
        self.sequence_len = sequence_len
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super().get_config()
        config.update({"sequence_len": self.sequence_len, "input_dim": self.input_dim, "output_dim": self.output_dim})
        return config

class TransformerEncoder(layers.Layer):
    def __init__(self, num_blocks, sequence_len, vocab_size, embed_dim, dense_dim, num_heads, dropout_rate=0.0, **kwargs):
        super().__init__(**kwargs)
        self.num_blocks = num_blocks
        self.sequence_len = sequence_len
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.dropout_rate = dropout_rate

        self.pos_embedding = PositionalEmbedding(sequence_len, vocab_size, embed_dim)
        block_layers = []
        for i in range(num_blocks):
            block_layers.append(TransformerEncoderBlock(embed_dim, dense_dim, num_heads, dropout_rate))
        self.block_layers = keras.Sequential(block_layers)

    def call(self, inputs):
        pos_inputs = self.pos_embedding(inputs)
        encoded = self.block_layers(pos_inputs)
        return encoded

    def get_config(self):
        config = super().get_config()
        config.update({
            'num_blocks': self.num_blocks, 'sequence_len': self.sequence_len,
            'vocab_size': self.vocab_size, 'embed_dim': self.embed_dim,
            'dense_dim': self.dense_dim, 'num_heads': self.num_heads,
            'dropout_rate': self.dropout_rate
        })
        return config

class TransformerDecoderBlock(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, dropout_rate=0.0, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.dropout_rate = dropout_rate

        self.attention_1 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim, dropout=dropout_rate)
        self.attention_2 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim, dropout=dropout_rate)
        self.dense_proj = keras.Sequential([
            layers.Dense(dense_dim, activation='relu'),
            layers.Dense(embed_dim)
        ])

        self.dropout1 = layers.Dropout(dropout_rate)
        self.dropout2 = layers.Dropout(dropout_rate)
        self.dropout3 = layers.Dropout(dropout_rate)

        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def get_config(self):
        config = super().get_config()
        config.update({"embed_dim": self.embed_dim, "dense_dim": self.dense_dim, "num_heads": self.num_heads, 'dropout_rate': self.dropout_rate})
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_len = input_shape[0], input_shape[1]
        i = tf.range(sequence_len)[:, tf.newaxis]
        j = tf.range(sequence_len)
        mask = tf.cast(i >= j, dtype='int32')
        mask = tf.reshape(mask, (1, sequence_len, sequence_len))
        mult = tf.concat([tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype='int32')
            padding_mask = tf.minimum(padding_mask, causal_mask)
        attention_output_1 = self.attention_1(query=inputs, key=inputs, value=inputs, attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(self.dropout1(attention_output_1)+inputs)
        attention_output_2 = self.attention_2(query=attention_output_1, key=encoder_outputs, value=encoder_outputs, attention_mask=padding_mask)
        attention_output_2 = self.layernorm_2(self.dropout2(attention_output_2) + attention_output_1)
        proj_output = self.dense_proj(attention_output_2)
        outputs = self.layernorm_3(attention_output_2 + self.dropout3(proj_output))
        return outputs

class TransformerDecoder(layers.Layer):
    def __init__(self, num_blocks, sequence_len, vocab_size, embed_dim, dense_dim, num_heads, dropout_rate=0.0, **kwargs):
        super().__init__(**kwargs)
        self.num_blocks = num_blocks
        self.sequence_len = sequence_len
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.dropout_rate = dropout_rate

        self.pos_embedding = PositionalEmbedding(sequence_len, vocab_size, embed_dim)
        self.block_layers = []
        for i in range(num_blocks):
            self.block_layers.append(TransformerDecoderBlock(embed_dim, dense_dim, num_heads, dropout_rate))
        self.dropout = layers.Dropout(dropout_rate)
        self.dense = layers.Dense(vocab_size, activation='softmax')

    def call(self, inputs, encoder_outputs):
        x = self.pos_embedding(inputs)
        for decoder_block in self.block_layers:
            x = decoder_block(x, encoder_outputs)
        x = self.dropout(x)
        outputs = self.dense(x)
        return outputs

    def get_config(self):
        config = super().get_config()
        config.update({
            'num_blocks': self.num_blocks, 'sequence_len': self.sequence_len,
            'vocab_size': self.vocab_size, 'embed_dim': self.embed_dim,
            'dense_dim': self.dense_dim, 'num_heads': self.num_heads,
            'dropout_rate': self.dropout_rate
            })
        return config

In [ ]:
transformer = keras.models.load_model('seq2seq_transformer.keras', custom_objects={'TransformerEncoderBlock': TransformerEncoderBlock, 'PositionalEmbedding': PositionalEmbedding,
                                    'TransformerEncoder': TransformerEncoder, 'TransformerDecoderBlock': TransformerDecoderBlock, 'TransformerDecoder': TransformerDecoder})
encoder = transformer.layers[2]
decoder = transformer.layers[3]
print(encoder.name, decoder.name)

transformer_encoder transformer_decoder


In [ ]:
sequence_len = 35

In [ ]:
import numpy as np

eng_vocab = english_vectorization.get_vocabulary()
eng_index_lookup = np.pad(np.array(eng_vocab), pad_width=(0,vocab_size), mode='constant', constant_values='[UNK]')[:vocab_size]
max_decoded_sentence_len = sequence_len
def decode_sequence(input_vectors):
    batch_size = input_vectors.shape[0]
    encoded_input_sentence = encoder.predict(input_vectors)
    decoded_sentence = np.repeat(np.array('[start]'), batch_size).reshape(-1, 1)
    blank = np.repeat(np.array(' '), batch_size).reshape(-1, 1)
    print(blank.shape)
    tokenized_target_sentence = english_vectorization(decoded_sentence)[:, :-1]
    tokenized_target_sentence = tf.Variable(initial_value=tokenized_target_sentence)

    for i in range(max_decoded_sentence_len):
        next_token_predict = decoder.predict([tokenized_target_sentence, encoded_input_sentence])
        # sampled_token_index = tf.random.categorical(next_token_predict[:, i], 1)
        sampled_token_index = np.argmax(next_token_predict[:, i],axis=1).reshape(-1, 1)
        sampled_token = eng_index_lookup[sampled_token_index]
        if i < max_decoded_sentence_len-1:
            tokenized_target_sentence[:, i+1, tf.newaxis].assign(sampled_token_index)
        decoded_sentence = np.char.add(decoded_sentence, np.char.add(blank, sampled_token))
        if sampled_token == '[end]':
            break
    return decoded_sentence

이곳에 번역할 한글 문장을 입력.

In [ ]:
korean_sentence = "옷이 다 찢어져 있으니 좀 새로 사 입어라."

preprocessed_korean_sentence = ' '.join(okt.morphs(korean_sentence))
print(preprocessed_korean_sentence)
korean_vector = korean_vectorization(preprocessed_korean_sentence)
print(korean_vector)

옷 이 다 찢어져 있으니 좀 새로 사 입어라 .
tf.Tensor(
[1365    2   43    1 2803  519 1621  200    1    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0], shape=(35,), dtype=int64)


In [ ]:
translated_sentence = decode_sequence(korean_vector[tf.newaxis, :])

1/1 [==============================] - 0s 78ms/step
(1, 1)
1/1 [==============================] - 0s 90ms/step


In [ ]:
print(re.sub("\[start\]|\[end\]", "", translated_sentence[0][0]).strip())

the clothes are all ripped so ill buy a new one
